In [1]:
from model5LHzc import *
from data_1ch import *

vid="v001_5" #version id should match the file number 
# The final number can be [1:5] and denote the cross-validation order
# Ran DellWS with GeForce RTX3060 GPU

In [2]:
LR=1e-3
loss=dice_loss
model = unet(LR, loss)
testGene = testGenerator("afm_cell_512/test/predh")
model.load_weights("files/unet_cell_seg_"+vid+".hdf5")
results = model.predict_generator(testGene,4,verbose=1)
saveResult("afm_cell_512/test/predh",results,vid[:4])

C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


4/4 [==============================] - 13s 56ms/step


### Train with data generator

In [2]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger

#Data augmentation
data_gen_args = dict(rotation_range=45,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='wrap')

# SETTINGS ***
batch_size=3
LR=1e-3

train_gen = trainGenerator(batch_size,'afm_cell_512/train0'+vid[-1],'afmheight','labels',data_gen_args,save_to_dir = None)
valid_gen = trainGenerator(batch_size,'afm_cell_512/valid0'+vid[-1],'afmheight','labels',data_gen_args,save_to_dir = None)

# 24 images are used for training, 6 images for validating and 4 images for testing
train_steps = 24//batch_size
valid_steps = 6//batch_size

# SETTINGS ***
loss=dice_loss
steps_per_epoch=20*train_steps
num_epochs=100

model = unet(LR, loss)

opt = tf.keras.optimizers.Adam(LR)
metrics = ["acc", dice_coef]
model.compile(loss=dice_loss, optimizer=opt, metrics=metrics)

callbacks = [
             ModelCheckpoint('files/unet_cell_seg_'+vid+'.hdf5', verbose=1, save_best_model=True),
             ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-8),
             CSVLogger("files/data_"+vid+".csv"),
             EarlyStopping(monitor="val_loss", patience=5, verbose=1)
            ]

model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=steps_per_epoch, validation_steps=valid_steps, 
                    epochs=num_epochs, callbacks=callbacks)


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Found 24 images belonging to 1 classes.
Found 24 images belonging to 1 classes.
Epoch 1/100
160/160 [==============================] - ETA: 0s - loss: 0.2421 - acc: 0.9231 - dice_coef: 0.7579Found 6 images belonging to 1 classes.
Found 6 images belonging to 1 classes.
160/160 [==============================] - 50s 233ms/step - loss: 0.2421 - acc: 0.9231 - dice_coef: 0.7579 - val_loss: 0.8693 - val_acc: 0.7344 - val_dice_coef: 0.1307

Epoch 00001: saving model to files\unet_cell_seg_v001_5.hdf5
Epoch 2/100
160/160 [==============================] - 36s 225ms/step - loss: 0.1296 - acc: 0.9500 - dice_coef: 0.8704 - val_loss: 0.7019 - val_acc: 0.7880 - val_dice_coef: 0.2981

Epoch 00002: saving model to files\unet_cell_seg_v001_5.hdf5
Epoch 3/100
160/160 [==============================] - 36s 227ms/step - loss: 0.0762 - acc: 0.9688 - dice_coef: 0.9238 - val_loss: 0.1878 - val_acc: 0.9395 - val_dice_coef: 0.8122

Epoch 00003: saving model to files\unet_cell_seg_v001_5.hdf5
Epoch 4/100
160/1

### validate your model and save predicted results

In [3]:
validGene = testGenerator("afm_cell_512/valid0"+vid[-1]+"/predh")
model.load_weights("files/unet_cell_seg_"+vid+".hdf5")
results = model.predict_generator(validGene,6,verbose=1)
saveResult("afm_cell_512/valid0"+vid[-1]+"/predh",results,vid[:4])

C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


6/6 [==============================] - 2s 73ms/step


In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 16) 160         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 512, 512, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 512, 512, 16) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [5]:
'''
testGene = testGenerator("afm_cell_512/test/predh")
model = tf.keras.models.load_model("files/unet_cell_seg_"+vid+".hdf5",custom_objects={'dice_loss': dice_loss,'dice_coef':dice_coef})
results = model.predict_generator(testGene,4,verbose=1)
saveResult("afm_cell_512/test/predh",results,vid)
'''

'\ntestGene = testGenerator("afm_cell_512/test/predh")\nmodel = tf.keras.models.load_model("files/unet_cell_seg_"+vid+".hdf5",custom_objects={\'dice_loss\': dice_loss,\'dice_coef\':dice_coef})\nresults = model.predict_generator(testGene,4,verbose=1)\nsaveResult("afm_cell_512/test/predh",results,vid)\n'